In [65]:
import urllib
import sqlalchemy
import numpy as np
import pandas as pd
from datetime import datetime

import os
import pandas as pd
import numpy as np


from helpers import rutas, validarProduccion
rutaRelativa        = rutas.rutas()
print(rutaRelativa)
estadoProduccion    = validarProduccion.validarProduccion()


from classes import dbAzure, email
dbAzure = dbAzure.DBAzure(estadoProduccion)
Email   = email.Email

print(estadoProduccion)

/Users/camilo/Desktop/SRC
CONEXION CON EL SERVIDOR
True


In [60]:
def queryCreateTable(conection_DB, nombre_tabla, dataframe):
    
    # ELIMINA BASE DE DATOS
    conection_DB.connection.execute(f'DROP TABLE GestionProcesosTienda.dbo.{nombre_tabla};')
    conection_DB.connection.commit()
    
    # CREA ARREGLO CON LAS COLUMNAS Y SUS TIPOS
    columns = []
    for index in dataframe.dtypes.index:
        tipo = str(dataframe.dtypes.loc[index])
        if tipo == "object":
            tipo = 'varchar'
        elif tipo == "int64":
            tipo = 'int'
        elif tipo == "float64":
            tipo = 'float'
        columns.append({'name': index, 'type': tipo})
    
    print(columns)
    
    # DA FORMATO A QUERY PARA CREAR TABLA 
    query = f"CREATE TABLE GestionProcesosTienda.dbo.{nombre_tabla} (\nid bigint IDENTITY(1,1) NOT NULL,"
    for column in columns:
        if column['type'] == 'varchar':
            query += f'\n{column["name"]} varchar(100) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL,'
        elif column['type'] == 'int':
            query += f'\n{column["name"]} bigint NOT NULL,'
    query += f"\ncreated_at datetime NOT NULL, \nupdated_at datetime NOT NULL, \nCONSTRAINT PK_{nombre_tabla}_3213E83F1717089E PRIMARY KEY (id)\n);"

    # SI LA TABLA NO EXISTE EN LA BASE DE DATOS LA CREA
    tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
    if 'productos_retirados' not in tablas.TABLE_NAME.unique():
        # CREA TABLA
        conection_DB.connection.execute(query)
        conection_DB.connection.commit()

        # AGREGA LAS COLUMNAS DE FECHA DE CREACION Y ACTUALIZACION
        dataframe['created_at']   = datetime.today()
        dataframe['updated_at']   = datetime.today()

        # INSERTA DATOS EN BASE DE DATOS
        db_params               = urllib.parse.quote_plus(conection_DB.dataConnection)
        engine                  = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params))
        dataframe.to_sql('productos_retirados', con=engine, index=False, if_exists="append", schema="dbo")

In [66]:
display(f'{rutaRelativa}/ROTACION/RETIRADOS.cvs')

'/Users/camilo/Desktop/SRC/ROTACION/RETIRADOS.cvs'

In [71]:
dataframe = pd.read_csv(f'{rutaRelativa}/ROTACION/RETIRADOS.csv')
dataframe = dataframe.head(10)
#dataframe = dataframe.drop(['EAN','SECCION','ALMACENAMIENTO','Fecha Merma','SECCION','N° Doc. Merma','ZONA','Creado por','Modificado','Creado','NO_CONFORMIDAD','CANTIDAD','ESTADO_RETIRO','FECHA_RETIRO','DESTINO','MES','ID_SECCION','LIMPIEZA','FECHA_SOLICITUD','PROVEEDOR','UBICACION','FECHA_VENCIMIENTO' ], axis=1)
dataframe = dataframe.drop(['Creado por', 'N° Doc. Merma', 'Fecha Merma'], axis=1)
dataframe = dataframe.replace({np.nan: 'None'})
dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: pd.Series(str(x).split(".")[0]))
dataframe['ID_SECCION'] = dataframe["ID_SECCION"].apply(lambda x: '0' + x if(int(x) < 10) else x)
display(dataframe)
dataframe.dtypes

,EAN,LOCAL,SECCION,DESCRIPCION,ID_SECCION,LIMPIEZA,FECHA_SOLICITUD,PROVEEDOR,UBICACION,FECHA_VENCIMIENTO,NO_CONFORMIDAD,CANTIDAD,ESTADO_RETIRO,FECHA_RETIRO,DESTINO,MES,ALMACENAMIENTO,Creado,Modificado,ZONA
0,7801916002880,J520 - JUMBO VALPARAISO,Fiambreria,TOCINO LA PREFERIDA 18G,02,CONFORME,24/06/2021,CONSORCIO IND. DE ALIMENTOS S.A. (C,None,30/07/2021,Sin Sato o Tarja,25,INMEDIATO,24/06/2021,DEVOLUCIÓN,None,None,24/06/2021 14:23,20/05/2022 11:45,None
1,7801916002880,J520 - JUMBO VALPARAISO,Fiambreria,TOCINO LA PREFERIDA 18G,02,CONFORME,24/06/2021,CONSORCIO IND. DE ALIMENTOS S.A. (C,None,28/06/2021,No Cumple MVU / TD,11,INMEDIATO,24/06/2021,DEVOLUCIÓN,None,None,24/06/2021 14:29,20/05/2022 11:45,None
2,77902085,J520 - JUMBO VALPARAISO,Quesería,QUESO STA.ROSA AZUL 1 GR,25,CONFORME,24/06/2021,SANTA ROSA CHILE ALIMENTOS LTDA.,Vitrina,29/06/2021,No Cumple MVU / TD,30,INMEDIATO,24/06/2021,MERMA,None,None,24/06/2021 17:43,24/06/2021 17:43,None
3,8421328020112,J520 - JUMBO VALPARAISO,Fábrica de Fiambre,LONGANIZA PARRILLERA ALEJANDRO 25G,36,CONFORME,25/06/2021,CECINAS DE SANTIAGO S.A.,Vitrina,08/08/2021,None,2,FECHA FUTURA,10/08/2021,MERMA,6.0,None,25/06/2021 12:15,25/06/2021 12:15,None
4,7801915000146,J520 - JUMBO VALPARAISO,SECCION FIAMBRERIA,SALCHICHA CERDO LA CRIANZA 250G,02,CONFORME,25/06/2021,AGROSUPER COM. DE ALIMENTOS LTDA.,Vitrina,07/07/2021,Tarja o Sato incompleto,15,INMEDIATO,25/06/2021,MERMA,6.0,None,25/06/2021 16:15,20/05/2022 11:45,None
5,7809611716071,J520 - JUMBO VALPARAISO,SECCION FIAMBRERIA,LONGANICILLA PATAGONICA LC 280G,02,CONFORME,25/06/2021,AGROSUPER COM. DE ALIMENTOS LTDA.,Cámara Refrigerada,30/06/2021,Con Pérdida de Vacío,5,HOY,25/06/2021,SIN STOCK,6.0,None,25/06/2021 16:35,20/05/2022 11:45,None
6,7809611707758,J520 - JUMBO VALPARAISO,SECCION FIAMBRERIA,LONGANICILLA MERKEN LA CRIANZA 280G,02,CONFORME,25/06/2021,AGROSUPER COM. DE ALIMENTOS LTDA.,Mural,16/07/2021,Con Pérdida de Vacío,13,INMEDIATO,25/06/2021,MERMA,6.0,None,25/06/2021 17:18,20/05/2022 11:45,None
7,7809611717320,J520 - JUMBO VALPARAISO,SECCION FIAMBRERIA,CHORIZO TRADICIONAL 240G LA CRIANZA,02,CONFORME,26/06/2021,AGROSUPER COM. DE ALIMENTOS LTDA.,Góndolas,16/07/2021,Sin Sato o Tarja,45,HOY,26/06/2021,MERMA,6.0,None,26/06/2021 12:42,20/05/2022 11:45,None
8,7801930006376,J520 - JUMBO VALPARAISO,Fiambreria,CHORIZO PARRILLERO PF 1K,02,CONFORME,29/06/2021,PRODUCTOS FERNANDEZ S A,Mural,15/07/2021,No Cumple MVU / TD,4,HOY,29/06/2021,DEVOLUCIÓN,6.0,None,29/06/2021 9:45,20/05/2022 11:45,None
9,7804617470675,J520 - JUMBO VALPARAISO,Quesería,QUESO MANTECOSO SURLAT TZO 5 GR,25,NO CONFORME,29/06/2021,SURLAT INDUSTRIAL S.A.,Bodega,30/06/2021,Tarja o Sato incompleto,12,INMEDIATO,29/06/2021,DEVOLUCIÓN,6.0,None,29/06/2021 9:56,29/06/2021 9:56,None


EAN                  object
LOCAL                object
SECCION              object
DESCRIPCION          object
ID_SECCION           object
LIMPIEZA             object
FECHA_SOLICITUD      object
PROVEEDOR            object
UBICACION            object
FECHA_VENCIMIENTO    object
NO_CONFORMIDAD       object
CANTIDAD             object
ESTADO_RETIRO        object
FECHA_RETIRO         object
DESTINO              object
MES                  object
ALMACENAMIENTO       object
Creado               object
Modificado           object
ZONA                 object
dtype: object

In [63]:
# EJECUTA FUNCION
nombre_tabla = 'productos_retirados'
queryCreateTable(dbAzure, nombre_tabla, dataframe)

[{'name': 'EAN', 'type': 'varchar'}, {'name': 'LOCAL', 'type': 'varchar'}, {'name': 'SECCION', 'type': 'varchar'}, {'name': 'DESCRIPCION', 'type': 'varchar'}, {'name': 'ID_SECCION', 'type': 'varchar'}, {'name': 'LIMPIEZA', 'type': 'varchar'}, {'name': 'FECHA_SOLICITUD', 'type': 'varchar'}, {'name': 'PROVEEDOR', 'type': 'varchar'}, {'name': 'UBICACION', 'type': 'varchar'}, {'name': 'FECHA_VENCIMIENTO', 'type': 'varchar'}, {'name': 'NO_CONFORMIDAD', 'type': 'varchar'}, {'name': 'CANTIDAD', 'type': 'varchar'}, {'name': 'ESTADO_RETIRO', 'type': 'varchar'}, {'name': 'FECHA_RETIRO', 'type': 'varchar'}, {'name': 'DESTINO', 'type': 'varchar'}, {'name': 'MES', 'type': 'varchar'}, {'name': 'ALMACENAMIENTO', 'type': 'varchar'}, {'name': 'Creado', 'type': 'varchar'}, {'name': 'Modificado', 'type': 'varchar'}, {'name': 'ZONA', 'type': 'varchar'}]


/var/folders/rs/5wl8vzfj4vz7xxdjjr9hk3800000gn/T/ipykernel_84143/566717003.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablas = pd.read_sql_query("select * from information_schema.tables", conection_DB.connection)
